In [33]:
from Parameters.TrainingParameters import TrainingParameters
from Utils.PrintUtils import print_big
from CustomExceptions.Exceptions import NotSupportedError
from json import load
from typing import Iterable, Union
import torch
import pandas as pd
from torch.jit import Error
from torch.utils.data import Dataset
import pm4py
from datetime import timedelta
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from Utils.Constants import Constants
from Utils.FileUtils import file_exists
import json
import os
from Parameters.Enums import PreprocessedDfType

In [34]:
filePath = TrainingParameters.bpi_2012_path

In [35]:
log = pm4py.read_xes(filePath)

parsing log, completed traces :: 100%|██████████| 13087/13087 [00:09<00:00, 1437.72it/s]


In [36]:
flattern_log: list[dict[str, any]]  = ([{**event,
                                                'caseid': trace.attributes['concept:name']}
                                               for trace in log for event in trace])
df = pd.DataFrame(flattern_log)

In [37]:
df["name_and_transition"] = df["concept:name"] + \
            "_" + df["lifecycle:transition"]
df = df[['time:timestamp', 'name_and_transition', "caseid"]]

In [38]:
df.head(5)

,time:timestamp,name_and_transition,caseid
0,2011-10-01 00:38:44.546000+02:00,A_SUBMITTED_COMPLETE,173688
1,2011-10-01 00:38:44.880000+02:00,A_PARTLYSUBMITTED_COMPLETE,173688
2,2011-10-01 00:39:37.906000+02:00,A_PREACCEPTED_COMPLETE,173688
3,2011-10-01 00:39:38.875000+02:00,W_Completeren aanvraag_SCHEDULE,173688
4,2011-10-01 11:36:46.437000+02:00,W_Completeren aanvraag_START,173688


In [54]:
newData = []
for case, group in df.groupby('caseid'):
            group.sort_values("time:timestamp", ascending=True, inplace=True)
            strating_time = group.iloc[0]["time:timestamp"] - \
                timedelta(microseconds=1)
            ending_time = group.iloc[-1]["time:timestamp"] + \
                timedelta(microseconds=1)

            # print("Before to dict")
            traces = group.to_dict('record')
            # print("After to dict")


            # Add start and end tags.
            traces.insert(
                0, {"caseid": case, "time:timestamp": strating_time, "name_and_transition": Constants.SOS_VOCAB})
            traces.append(
                {"caseid": case, "time:timestamp": ending_time, "name_and_transition": Constants.EOS_VOCAB})
            # print("before extend to dict")
            newData.extend(traces)
            # print("After extend to dict")



/home/jrhs/.pyenv/versions/dl/lib/python3.9/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [55]:

df = pd.DataFrame(newData)

In [53]:
df.head(5)

,caseid,time:timestamp,name_and_transition
0,173688,2011-10-01 00:38:44.545998+02:00,<SOS>
1,173688,2011-10-01 00:38:44.545999+02:00,<SOS>
2,173688,2011-10-01 00:38:44.546000+02:00,A_SUBMITTED_COMPLETE
3,173688,2011-10-01 00:38:44.880000+02:00,A_PARTLYSUBMITTED_COMPLETE
4,173688,2011-10-01 00:39:37.906000+02:00,A_PREACCEPTED_COMPLETE


In [49]:
another_df = pd.DataFrame(newData)

In [50]:
another_df.head(5)

,caseid,time:timestamp,name_and_transition
0,173688,2011-10-01 00:38:44.545998+02:00,<SOS>
1,173688,2011-10-01 00:38:44.545999+02:00,<SOS>
2,173688,2011-10-01 00:38:44.546000+02:00,A_SUBMITTED_COMPLETE
3,173688,2011-10-01 00:38:44.880000+02:00,A_PARTLYSUBMITTED_COMPLETE
4,173688,2011-10-01 00:39:37.906000+02:00,A_PREACCEPTED_COMPLETE
